In [1]:
from z3 import *
z3.set_param("proof", True)

In [2]:
a, b = Int("a"), Int("b")
Reach = Function("Reach", IntSort(), IntSort(), BoolSort())

chcs = [
    ForAll([a, b], Implies(And(a > 0, b > 0), Reach(a, b))),
    ForAll([a, b], Implies(Reach(a, b), Reach(b, a + b))),
    ForAll([a, b], Implies(Reach(a, b), And(a > 0, b > 0)))
]

s = SolverFor("HORN")
s.add(chcs)
s.check()

sat

In [3]:
s.model()

[Reach = [else -> And(Not(Var(1) <= 0), Not(Var(0) <= 0))]]

In [4]:
a, b = Int("a"), Int("b")
a_next, b_next = Int("a'"), Int("b'")
Reach = Function("Reach", IntSort(), IntSort(), BoolSort())

chcs = [
    ForAll([a, b], Implies(And(a > 0, b > 0), Reach(a, b))),
    ForAll([a, b, a_next, b_next], 
           Implies(And(Reach(a, b), a_next == b, b_next == a),
                   Reach(a_next, b_next))),
    ForAll([a, b], Implies(Reach(a, b), And(a > 0, b > 0)))
]

s = SolverFor("HORN")
s.add(chcs)
s.check()

sat

In [5]:
s.model()

[Reach = [else -> And(Not(Var(1) <= 0), Not(Var(0) <= 0))]]

In [6]:
import sys
sys.path.append("../../hw/base")

from verification_utils import CHCs, mk_int_array

In [7]:
chcs = CHCs([
    Implies(And(a > 0, b == 0), Reach(a, b)),
    Implies(And(Reach(a, b), a_next == b + 1, b_next == a + b),
            Reach(a_next, b_next)),
    Implies(Reach(a, b), And(a > 0, b >= 0))
])

chcs.solve()

Reach,¬(ν0 ≤ 0) ∧ ¬(ν1 ≤ -1)


In [19]:
from verification_utils import CHCs
'''
for (i = 0; i < n; i++) {
    a[i] = n;
}
'''

a, n, i, a_next, n_next, i_next = Ints("a n i a' n' i'")
mem = Array("mem", IntSort(), IntSort())
mem_next = Array("mem'", IntSort(), IntSort())

j = Int("j")

# Inv(a, n, i, mem)
Inv = Function("Inv", a.sort(), n.sort(), i.sort(), mem.sort(), j.sort(), BoolSort())

chcs = CHCs([
    Implies(And(n > 0, j >= 0, j < n), Inv(a, n, 0, K(mem.domain(), 0), j)),
    Implies(And(Inv(a, n, i, mem, j), i < n), Inv(a, n, i + 1, Store(mem, i, n), j)),
    ##--- Diverges! Use ghost instead
    #Implies(And(Inv(a, n, i, mem, j), Not(i < n)), ForAll([i], Implies(And(i < n, i >= 0), mem[i] == n))),
    Implies(And(Inv(a, n, i, mem, j), Not(i < n)), mem[j] != 0)
])
chcs

"n > 0 ∧ j ≥ 0 ∧ j < n ⇒ Inv(a, n, 0, K(Int, 0), j)"
"Inv(a, n, i, mem, j) ∧ i < n ⇒ Inv(a, n, i + 1, Store(mem, i, n), j)"
"Inv(a, n, i, mem, j) ∧ ¬(i < n) ⇒ mem[j] ≠ 0"


In [20]:
sol = chcs.solve({'fp.spacer.global': True})
sol

Inv,¬(ν4 + -1·ν1 ≥ 0) ∧ ¬(ν1 ≤ 0) ∧ (¬(ν3[ν4] ≤ 0) ∨ ν4 = ν2 ∨ ¬(ν4 + -1·ν2 ≤ 0))
